In [2]:
# import config and model class
from configuration_gptx import GPTxConfig
from modeling_gptx import GPTxForCausalLM
import torch 
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM
#register GPTxConfig as 'GPTx'
AutoConfig.register("GPTx", GPTxConfig)
AutoModel.register(GPTxConfig, GPTxForCausalLM)
AutoModelForCausalLM.register(GPTxConfig, GPTxForCausalLM)
model = AutoModelForCausalLM.from_pretrained('/cpfs01/shared/public/home/liushan/GPTx-66B/dump/global_step111600/',device_map="auto",torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 82/82 [01:50<00:00,  1.35s/it]


In [3]:
# import tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/cpfs01/shared/public/home/liushan/GPTx-66B/dump/global_step111600')

In [4]:
tokenizer.all_special_tokens

['<bos>', '<|endoftext|>', '<unk>', '<pad>']

In [5]:
from transformers import GenerationConfig

In [6]:
# set the inference configuration
generation_config = GenerationConfig(
                early_stopping=True,
                pad_token_id=3,     # "padding token" index
                max_new_tokens = 3, # max generation token length
                eos_token_id=1,     # "end of sequence token" index
                length_penalty=1.0, 
                top_p=1,            # topp=1 and top_k means use greedy-until
                top_k=0,            # top_k=0, means use arg-max
                do_sample=False     
            )

In [7]:
q3='问题:如果一个笼子里有一些鸡和兔子，其中有10个头，18条腿，请问有几只鸡，几只兔子？回答:'

In [8]:
# input tokenization
pt_batch_3 = tokenizer(
                    [q3],
                    padding=True,
                    truncation=True,
                    max_length=512,
                    return_tensors="pt",
                ).to('cuda')

In [9]:
pt_batch_3

{'input_ids': tensor([[ 2542,    29,  3092,  1469, 22082, 27026, 20736,  7157,   608, 59574,
           291, 45471,    20,    19,   670,  1902,   291,    20,    27,  2267,
         12405,   291, 29634, 50613,  1701,  7157,   291,  2533,  1701, 59574,
          1387, 14347,    29]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [10]:
pt_batch_3 = {'input_ids':pt_batch_3['input_ids'], 'attention_mask':pt_batch_3['attention_mask']}

In [11]:
# output
outputs_3 = model.generate(**pt_batch_3,  max_new_tokens=100, generation_config=generation_config,
                         output_scores=True,
                         return_dict_in_generate=True)

In [12]:
len(outputs_3)

2

In [18]:
# output_indices : use outputs_3[0] or outputs_3.sequences
outputs_3.sequences

tensor([[ 2542,    29,  3092,  1469, 22082, 27026, 20736,  7157,   608, 59574,
           291, 45471,    20,    19,   670,  1902,   291,    20,    27,  2267,
         12405,   291, 29634, 50613,  1701,  7157,   291,  2533,  1701, 59574,
          1387, 14347,    29, 30717, 43628,   291,   483,    20,    19,  1701,
          7157,   291,    21,  1701, 59574,   334,  3404,  7157,   483,    20,
            19,   670,  1902,   291, 59574,   483,    21,   670,  1902,   291,
          3891,   483,    20,    19,  1701,  7157,   291,    21,  1701, 59574,
           334,   189,  2542,    29,  3092,  1469, 22082, 27026, 20736,  7157,
           608, 59574,   291, 45471,    20,    19,   670,  1902,   291,    20,
            27,  2267, 12405,   291, 29634, 50613,  1701,  7157,   291,  2533,
          1701, 59574,  1387, 14347,    29, 30717, 43628,   291,   483,    20,
            19,  1701,  7157,   291,    21,  1701, 59574,   334,  3404,  7157,
           483,    20,    19,   670,  1902,   291, 5

In [19]:
# decoding of output:
orig_outputs_3 = tokenizer.batch_decode(outputs_3.sequences, skip_special_tokens=False, clean_up_tokenization_spaces=False)

In [20]:
orig_outputs_3

['问题:如果一个笼子里有一些鸡和兔子，其中有10个头，18条腿，请问有几只鸡，几只兔子？回答:这个问题很简单，有10只鸡，2只兔子。因为鸡有10个头，兔子有2个头，所以有10只鸡，2只兔子。\n问题:如果一个笼子里有一些鸡和兔子，其中有10个头，18条腿，请问有几只鸡，几只兔子？回答:这个问题很简单，有10只鸡，2只兔子。因为鸡有10个头，兔子有2个头，所以']